In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
from io import StringIO
from bs4 import BeautifulSoup
import contextlib
from selenium.common.exceptions import InvalidArgumentException
import time
import pandas as pd
from datetime import datetime
pd.set_option('display.max_rows', 1200)

# ignore warnings
pd.options.mode.chained_assignment = None

import undetected_chromedriver as uc

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth

In [ ]:
def start_driver():
    driver = uc.Chrome(use_subprocess=True, version_main=133)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.maximize_window()
    return driver

In [ ]:
# start webdriver
driver = start_driver()

# Log In

In [ ]:
def depop_login(driver, username, password):
    # go to depop login page
    driver.get('https://www.depop.com/login/')
    
    # click "Accept" cookies button if it pops up
    try:
        driver.find_element(By.XPATH, '/html/body/div/div/div[3]/div[2]/button[2]').click()
    except:
        pass

    # click continue w/ email
    try:
        driver.find_element(By.XPATH, ('//span[contains(text(),"Continue with email")]')).click()
        time.sleep(1)
    except:
        pass

    # enter username 
    driver.find_element(By.XPATH, "//*[contains(@name, 'email')]").send_keys(username)
    time.sleep(1)

    # click continue
    driver.find_element(By.XPATH, "//*[contains(@type, 'submit')]").click()
    time.sleep(1)

    # ask for magic email link
    email_link = input('Enter the Depop login URL sent via email: ')
    driver.get(email_link)

In [ ]:
depop_login(driver, 'username', 'password')

# Get Current Products

In [ ]:
# go to selling hub
driver.get('https://www.depop.com/sellinghub/selling/')

In [ ]:
# scroll down to bottom while allowing time for products to load (gets page source every loop to deal w/ depop loading error)
def scrolltobottom(driver):
    curr_scrollheight = driver.execute_script("return document.documentElement.scrollHeight")
    for i in range(0,100):
        page = driver.find_element("tag name","html")
        page.send_keys(Keys.END)
        time.sleep(2)
        new_scrollheight = driver.execute_script("return document.documentElement.scrollHeight")
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        if new_scrollheight != curr_scrollheight:
            curr_scrollheight = new_scrollheight
        else:
            return driver.page_source, BeautifulSoup(page_source, 'lxml')
            break

In [ ]:
page_source, soup = scrolltobottom(driver)

In [ ]:
# empty vectors for variable storage
links = []
product_names = []
prices = []
months = []
days = []
years = []
dates = []
sold_markers = []
boost_markers = []
discount_markers = []
discount_prices = []
current_discounts = []

# dictionary for month string conversion
month_dict ={'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}

# get product names
name_soup = soup.find_all("span",attrs={"class":"_text_bevez_41 _shared_bevez_6 _normal_bevez_51 styles_description__O5n9J styles_overflowText__T0sDd"})

for tag in name_soup:
    p_name = re.split(' - ', tag.text)[0]
    p_name = re.split('\n', p_name)[0]
    # print(p_name)
    product_names.append(p_name)
    
# get dates
date_soup = soup.find_all("p",attrs={"class":'_text_bevez_41 _shared_bevez_6 _normal_bevez_51 _caption1_bevez_55 styles_overflowText__T0sDd'})

for tag in date_soup:
    try:
        day, month, year = list(filter(None, re.split('Listed on | ', tag.text)))
        days.append(int(day))
        months.append(month_dict[month])
        years.append(int(year))
        date = str(month_dict[month])+'-'+str(day)+'-'+str(year)
        dates.append(date)
        # print(date)
    except:
        day, month, year = list(filter(None, re.split('Last updated on | ', tag.text)))
        days.append(int(day))
        months.append(month_dict[month])
        years.append(int(year))
        date = str(month_dict[month])+'-'+str(day)+'-'+str(year)
        dates.append(date)
        # print(date)
        
# get prices
price_soup = soup.find_all("div",attrs={"class":'styles_priceContainer__BPqEu'})

for tag in price_soup:
    # print(tag.text)
    if tag.text.count("$") == 2:
        # print(re.split(r'US\$', tag.text))
        discount_markers.append(1)
        discount_price = float(re.split(r'US\$', tag.text)[-1])
        discount_prices.append(discount_price)
        og_price = float(re.split(r'US\$', tag.text)[1])
        prices.append(og_price)
        curr_discount = round(((og_price - discount_price) / og_price) * 100)
        current_discounts.append(curr_discount)
    else:
        discount_markers.append(0)
        discount_prices.append(float(re.split(r'US\$', tag.text)[-1]))
        prices.append(float(re.split(r'US\$', tag.text)[-1]))
        current_discounts.append(0)
    
# get product links
link_soup = soup.find_all("div",attrs={"class":"styles_listingRow__ckmtm styles_listingRowCheckboxes__xdtqo"})

for tag in link_soup:
    href_link = tag.find('a').get('href')
    links.append('https://depop.com' + href_link)
    if 'Sold' in tag.text:
        sold_markers.append(1)
    else:
        sold_markers.append(0)
        
boost_soup = soup.find_all("div",attrs={"class":"styles_desktopInsights__XI2Vf"})

for tag in boost_soup:
    if "Boost to see" in tag.text:
        boost_markers.append(0)
    else:
        boost_markers.append(1)

In [ ]:
for x in [product_names, links, prices, dates, sold_markers, boost_markers, discount_markers, discount_prices, current_discounts]:
    print(len(x))

In [ ]:
# dataframe w/ prices and items
dict = {'Item': product_names, 'Link': links, 'Price': prices, 'Discount Price' : discount_prices, 'Last Updated': dates, 'Sold' : sold_markers, 'Boosted' : boost_markers, 'Discounted' : discount_markers, 'Discount %': current_discounts} 

df = pd.DataFrame(dict)
df['Last Updated'] = pd.to_datetime(df['Last Updated'], format='%m-%d-%Y')
df['Days Since Updating'] = (datetime.today()-df['Last Updated']).dt.days
display(df)

# Boost Listings

In [ ]:
# remove sold listings, listings posted within the last week, boosted listings
boost_df = df.loc[(df['Sold'] == 0) & (df['Days Since Updating'] >= 7) & (df['Boosted'] == 0)].reset_index(drop=True)

In [ ]:
print('Number of items to be boosted:', str(len(boost_df)))

In [ ]:
# boost loop
for i in range(0, len(boost_df)):
    if i==0:
        # click "Accept" cookies button if it pops up
        try:
            driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div[2]/button[2]').click()
        except:
            pass
    item = boost_df.iloc[i,list(boost_df.columns).index('Item')]
    link = boost_df.iloc[i,list(boost_df.columns).index('Link')]
        

    print('Item '+ str(i+1) + ' of ' + str(len(boost_df)), '('+item+')\n')
    driver.get(link)
    time.sleep(1)
    boost_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(@class, 'ButtonBoost')]")))
    boost_element.click()
    confirm_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(@class, 'ConfirmAndBoost')]")))
    confirm_element.click()
    time.sleep(1)
    
print('Done boosting listings.')

# Discount Non-Discounted Listings

In [ ]:
 discount_amt = int(input('Enter the a whole number percent (out of 100, divisible by 5) to discount all non-discounted listings by: '))

In [ ]:
# go to selling hub
driver.get('https://www.depop.com/sellinghub/selling/')

In [ ]:
# scroll down to bottom while allowing time for products to load (gets page source every loop to deal w/ depop loading error)
curr_scrollheight = driver.execute_script("return document.documentElement.scrollHeight")
for i in range(0,100):
    page = driver.find_element("tag name","html")
    page.send_keys(Keys.END)
    time.sleep(2)
    new_scrollheight = driver.execute_script("return document.documentElement.scrollHeight")
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    if new_scrollheight != curr_scrollheight:
        curr_scrollheight = new_scrollheight
    else:
        break

In [ ]:
# scroll back up
page = driver.find_element("tag name","html")
page.send_keys(Keys.HOME)

In [ ]:
checkboxes = driver.find_elements(By.XPATH, ("//input[contains(@class, '_checkbox')]"))[1:]

In [ ]:
len(checkboxes)

In [ ]:
df['Checkbox'] = checkboxes
df['Checkbox_x'] = [x.location['x'] for x in checkboxes]
df['Checkbox_y'] = [x.location['y'] for x in checkboxes]

In [ ]:
# remove sold listings, listings posted within the last week, discounted listings
discount_df = df.loc[(df['Sold'] == 0) & (df['Days Since Updating'] >= 7) & (df['Discounted'] == 0)].reset_index(drop=True)

In [ ]:
display(discount_df)

In [ ]:
tick_dict = {
    5:0,
    10:1,
    15:2,
    20:3,
    25:4,
    30:5,
    35:6,
    40:7,
    45:8,
    50:9,
    55:10,
    60:11,
    65:12,
    70:13,
    75:14,
    80:15,
    85:16,
    90:17,
    95:18,
}

In [ ]:
# loop through discount_df checkboxes and click them
itemset = 1
for i in range(len(discount_df)):
    # for first item
    if i==0:
        checkbox = discount_df.at[i, 'Checkbox']
        scroll_by_coord = 'window.scrollTo(%s,%s);' % (
                checkbox.location['x'],
                checkbox.location['y']
            )
        scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
        driver.execute_script(scroll_by_coord)
        driver.execute_script(scroll_nav_out_of_way)
        print('scrolled to and clicked checkbox for', discount_df.at[i, 'Item'], '(Item ' + str(i+1) + ' of ' + str(len(discount_df)) + ')' )
        driver.execute_script("arguments[0].click();", checkbox)
    
    # if item number is not divisible by 10 and not the last item, click checkbox for discounting
    elif i%10 != 0 and i+1!=len(discount_df):
        checkbox = discount_df.at[i, 'Checkbox']
        scroll_by_coord = 'window.scrollTo(%s,%s);' % (
                checkbox.location['x'],
                checkbox.location['y']
            )
        scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
        driver.execute_script(scroll_by_coord)
        driver.execute_script(scroll_nav_out_of_way)
        print('scrolled to and clicked checkbox for', discount_df.at[i, 'Item'], '(Item ' + str(i+1) + ' of ' + str(len(discount_df)) + ')' )
        driver.execute_script("arguments[0].click();", checkbox)
    
    # if item number IS divisible by 10 and not the last item, click checkbox for discounting, discount, reset selected checkboxes    
    elif i%10 == 0 and i+1!=len(discount_df):
        checkbox = discount_df.at[i, 'Checkbox']
        scroll_by_coord = 'window.scrollTo(%s,%s);' % (
                checkbox.location['x'],
                checkbox.location['y']
            )
        scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
        driver.execute_script(scroll_by_coord)
        driver.execute_script(scroll_nav_out_of_way)
        print('scrolled to and clicked checkbox for', discount_df.at[i, 'Item'], '(Item ' + str(i+1) + ' of ' + str(len(discount_df)) + ')' )
        driver.execute_script("arguments[0].click();", checkbox)
        
        # after 10 listings are selected, discount then clear selections
        
        # click discount button
        driver.find_element(By.XPATH, ('//button[span="Discount"]')).click()

        # get slider element
        time.sleep(5)
        slider = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ('//input[@list="ticks"]'))))

        # move slider
        for i in range(tick_dict[discount_amt]):
            slider.send_keys(Keys.RIGHT)
    
        # click confirm
        time.sleep(1)
        driver.find_element(By.XPATH, ('//button[span="Confirm"]')).click()

        # CLICK x
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, "[aria-label='Close']").click()

        print('Discounted item set ' + str(itemset))
        itemset+=1
        
        # scroll back up
        time.sleep(3)
        page.send_keys(Keys.HOME)
    
    # handles last item (since the last item number is not likely to be evenly divisible by 10)
    else:
        checkbox = discount_df.at[i, 'Checkbox']
        scroll_by_coord = 'window.scrollTo(%s,%s);' % (
                checkbox.location['x'],
                checkbox.location['y']
            )
        scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
        driver.execute_script(scroll_by_coord)
        driver.execute_script(scroll_nav_out_of_way)
        print('scrolled to and clicked checkbox for', discount_df.at[i, 'Item'], '(Item ' + str(i+1) + ' of ' + str(len(discount_df)) + ')' )
        driver.execute_script("arguments[0].click();", checkbox)
        
        # click discount button
        driver.find_element(By.XPATH, ('//button[span="Discount"]')).click()

        # get slider element
        time.sleep(5)
        slider = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ('//input[@list="ticks"]'))))

        # move slider
        for i in range(tick_dict[discount_amt]):
            slider.send_keys(Keys.RIGHT)
    
        # click confirm
        time.sleep(1)
        driver.find_element(By.XPATH, ('//button[span="Confirm"]')).click()

        # CLICK x
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, "[aria-label='Close']").click()


# Discount Everything That Wasn't Just Discounted by 5% More

In [ ]:
tick_dict = {
    5:0,
    10:1,
    15:2,
    20:3,
    25:4,
    30:5,
    35:6,
    40:7,
    45:8,
    50:9,
    55:10,
    60:11,
    65:12,
    70:13,
    75:14,
    80:15,
    85:16,
    90:17,
    95:18,
}

In [ ]:
# create another dataframe that contains items that weren't discounted in the last step
discount_df2 = (df[df.merge(discount_df[['Item']], indicator=True, how='left')['_merge'].eq('left_only')].reset_index(drop=True))

In [ ]:
discount_df2 = discount_df2.loc[(discount_df2['Sold'] == 0) & (discount_df2['Days Since Updating'] >= 7)].reset_index(drop=True)

In [ ]:
discount_df2

In [ ]:
# scroll back up
page = driver.find_element("tag name","html")
page.send_keys(Keys.HOME)

In [ ]:
# select all and then deselect all to clear selections
driver.find_elements(By.XPATH, ("//input[contains(@class, '_checkbox')]"))[0].click()
time.sleep(5)
driver.find_element(By.XPATH, ("//*[contains(@class, '_tickIcon')]")).click()

In [ ]:
discount_df2

In [ ]:
def discount_everything(df):
    df_length = len(df)
    dataframe = df.copy()
    i = 0
    j = 0
    while True:
        try:
            driver.get('https://www.depop.com/sellinghub/selling/')
            page_source, soup = scrolltobottom(driver)
            page = driver.find_element("tag name","html")
            page.send_keys(Keys.HOME)
            listingRows = driver.find_elements(By.XPATH, ("//div[contains(@class, 'listingRow')]"))
            for itemname in dataframe['Item']:
                i+=1
                # get row for current item in loop
                for row in listingRows:
                    if itemname in row.text:
                        current_row = row
                # get checkbox to click later
                checkbox = current_row.children()[0]
                # determine if price is already discounted
                # if item has been discounted
                if current_row.text.count("$") == 2:
                    curr_price = float(re.split(r'US\$|\n', current_row.find_element(By.CSS_SELECTOR, "[class='styles_priceContainer__BPqEu']").text)[-1])
                    og_price = float(re.split(r'US\$|\n', current_row.find_element(By.CSS_SELECTOR, "[class='styles_priceContainer__BPqEu']").text)[-3])
                # if item has not been discounted
                else:
                    curr_price = float(re.split(r'US\$|\n', current_row.find_element(By.CSS_SELECTOR, "[class='styles_priceContainer__BPqEu']").text)[-1])
                    og_price = float(re.split(r'US\$|\n', current_row.find_element(By.CSS_SELECTOR, "[class='styles_priceContainer__BPqEu']").text)[-1])
                # calculate current discount
                curr_discount = round(((og_price - curr_price) / og_price) * 100)
                # do some scrolling, click checkbox
                scroll_by_coord = 'window.scrollTo(%s,%s);' % (
                        checkbox.location['x'],
                        checkbox.location['y']
                    )
                scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
                driver.execute_script(scroll_by_coord)
                driver.execute_script(scroll_nav_out_of_way)
                print('Scrolled to and clicked checkbox for', itemname, '(Item', str(i), 'of', str(df_length) + ')')
                driver.execute_script("arguments[0].click();", checkbox)
                # click discount button
                driver.find_element(By.XPATH, ('//button[span="Discount"]')).click()
                
                # get slider element after sleeping
                time.sleep(2.5)
                slider = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ('//input[@list="ticks"]'))))
                slider_value = int(slider.get_attribute('value'))
                
                # if the item has already been discounted
                if slider_value > 5 or curr_discount == 5:
                    # move slider to the right once, increasing the discount by 5%
                    slider.send_keys(Keys.RIGHT)
                    time.sleep(2.5)
                    # get new slider value
                    new_slider_value = int(slider.get_attribute('value'))
                    # click confirm
                    driver.find_element(By.XPATH, ('//button[span="Confirm"]')).click()
                    # wait for discount to process
                    time.sleep(2.5)
                    # close discount window
                    driver.find_element(By.CSS_SELECTOR, "[aria-label='Close']").click()
                    # print discount message
                    print('\tChanged discount percentage  of', itemname, 'from ' + str(slider_value) + ' to ' + str(new_slider_value))
                    
                # if the item has not been discounted. just need to click confirm to set a discount of 5%
                else:
                    # click confirm
                    driver.find_element(By.XPATH, ('//button[span="Confirm"]')).click()
                    # wait for discount to process
                    time.sleep(2.5)
                    # close discount window
                    driver.find_element(By.CSS_SELECTOR, "[aria-label='Close']").click()
                    # print discount message
                    print('\tChanged discount percentage  of', itemname, 'from ' + str(0) + ' to ' + str(slider_value))
                        
                # scroll back up
                time.sleep(3)
                page = driver.find_element("tag name","html")
                page.send_keys(Keys.HOME)

                # remove discounted item from copied dataframe
                dataframe = dataframe.tail(-1)
        except Exception as e:
            j+=1
            print(f"An error occurred: {e}. Restarting loop.")
            driver.get_screenshot_as_file('depop_exception_' + dt.date.today().strftime('%Y-%m-%d') + '.png')
            if j == 10:
                break
        else:
            break

In [ ]:
discount_everything(discount_df2)